In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas_ta as ta
import json
import requests
from yahoofinancials import YahooFinancials

### Get the Financial Features

In [50]:
ticker = 'BLK'
start_date = '2018-01-01'
end_date = '2023-10-01'

start_date_test = '2023-01-01'
end_date_test = '2023-10-01' 

#### Historical Stock Price

In [51]:
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start = start_date, end = end_date)
    return stock_data

In [52]:
sp500 = pd.read_csv('../data/sp500_stocks.csv')

#### Technical Indicators

Since prediction in the near future is highly dependent on momentum, we primarily consider momentum indicators:  
-> SMA is simple moving average which takes the average over a window (we use two values with window size of 50 and 200)  
-> MACD, short for moving average convergence/divergence, is a trading indicator designed to reveal changes in the strength, direction, momentum, and duration of a trend in a stock's price  
-> RSI or relative strength index is a momentum indicator that measures the magnitude of recent price changes  

In [53]:
def calculate_technical_indicators(stock_data):
    # moving averages
    stock_data['SMA_50'] = stock_data['Adj Close'].rolling(window=50).mean()
    stock_data['SMA_200'] = stock_data['Adj Close'].rolling(window=200).mean()
    
    # MACD
    stock_data.ta.macd(append=True)
    
    # RSI
    stock_data.ta.rsi(append=True)
    
    return stock_data

#### Volatility and Returns

In [54]:
def calculate_volatility(stock_data):
    stock_data['Volatility'] = stock_data['Adj Close'].pct_change().rolling(window=20).std()
    return stock_data
def calculate_returns(stock_data):
    stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change()
    return stock_data

#### Fundamentals

Stock price is often dependent on the market conditions at the time so we look at two major sources of fundamental data:  
-> CBOE Interest Rate 10 Year or the 10 year interest rate on Treasury Bonds  
-> Bond volatility rate index or VIX which is a good indicator of market volatility  

In [55]:
def get_bond_interest_rate(stock_data, start_date, end_date):
    interest_rate = yf.download('^TNX', start = start_date, end = end_date)
    stock_data['interest_rate'] = interest_rate['Adj Close'].tolist()
    return stock_data

def get_bond_volatility_rate(stock_data, start_date, end_date):
    vix = yf.download('^VIX', start=start_date, end=end_date)
    stock_data['vix'] = vix['Adj Close'].tolist()
    return stock_data

In [56]:
def highest_correlation(df, target_stock):
    target_prices = df[df['Symbol']==target_stock]['Adj Close']
    pivot_df = df.pivot(index='Date', columns='Symbol', values='Adj Close')
    corr_matrix = pivot_df.corr()
    top_3_corr = corr_matrix[target_stock].drop(target_stock).sort_values(ascending=False).head(3)
    result = top_3_corr.keys().tolist()
    result = [string.upper() for string in result]
    return result

In [59]:
print(highest_correlation(sp500, 'BLK'))

['AVY', 'LH', 'NXPI']


#### Supporting Data

Additionally, we want to see how index prices and similar stocks play an impact on the price of a particular stock. In this case, since we are using Apple stock as an example, we consider the price of similar tech stocks like Microsoft, Google, Amazon, and Facebook as Features. Additionally, Apple is listed on the NASDAQ so looking at the nasdaq value and the S&P500 could also be a useful feature

In [60]:
def get_similar_stocks(sp500, stock_data, start_date, end_date):
    symbols = highest_correlation(df = sp500, target_stock=ticker)
    for symbol in symbols:
        historical_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data[symbol.lower()] = historical_data['Adj Close']
    return stock_data
    
    
def get_index_data(stock_data, start_date, end_date):
    nsdq = yf.download('^IXIC', start=start_date, end=end_date)
    spy = yf.download('^GSPC', start=start_date, end=end_date)
    stock_data['nsdq'] = nsdq['Adj Close'].tolist()
    stock_data['spy'] = spy['Adj Close'].tolist()
    return stock_data

In [61]:
def get_quarterly_financials(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    quarterly_financials = ticker.quarterly_financials
    # Transpose the DataFrame for easier merging
    quarterly_financials_transposed = quarterly_financials.transpose()
    return quarterly_financials_transposed

#### Get the data

In [62]:
stock_data = get_stock_data(ticker=ticker, start_date=start_date, end_date=end_date)
stock_data = calculate_technical_indicators(stock_data)
stock_data = calculate_volatility(stock_data)
stock_data = calculate_returns(stock_data)
stock_data = get_bond_interest_rate(stock_data, start_date=start_date, end_date=end_date)
stock_data = get_bond_volatility_rate(stock_data, start_date=start_date, end_date=end_date)
stock_data = get_similar_stocks(sp500, stock_data, start_date=start_date, end_date=end_date)
stock_data = get_index_data(stock_data, start_date=start_date, end_date=end_date)


# stock_data_test = get_stock_data(ticker=ticker, start_date=start_date_test, end_date=end_date_test)
# stock_data_test = calculate_technical_indicators(stock_data_test)
# stock_data_test = calculate_volatility(stock_data_test)
# stock_data_test = calculate_returns(stock_data_test)
# stock_data_test = get_bond_interest_rate(stock_data_test, start_date=start_date, end_date=end_date)
# stock_data_test = get_bond_volatility_rate(stock_data_test, start_date=start_date, end_date=end_date)
# stock_data_test = get_similar_stocks(stock_data_test, start_date=start_date, end_date=end_date)
# stock_data_test = get_index_data(stock_data_test, start_date=start_date, end_date=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


#### Drop unused columns

In [63]:
stock_data = stock_data.drop(['High', 'Low', 'Close'], axis=1)
#stock_data_test = stock_data_test.drop(['High', 'Low', 'Close'], axis=1)

In [64]:
stock_data.to_csv('../data/stock_data_blk.csv')
#stock_data_test.to_csv('./data/stock_data_test.csv')

In [65]:
stock_data.head()

,Open,Adj Close,Volume,SMA_50,SMA_200,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,Volatility,Daily_Return,interest_rate,vix,avy,lh,nxpi,nsdq,spy
Date,,,,,,,,,,,,,,,,,,
2018-01-02,518.780029,432.673889,526400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.465,9.77,103.802498,135.998566,108.132973,7006.899902,2695.810059
2018-01-03,508.660004,437.238922,448700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010551,2.447,9.15,104.098618,136.627045,108.252136,7065.529785,2713.060059
2018-01-04,516.500000,443.010986,389800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013201,2.453,9.22,104.789566,136.132645,108.059624,7077.910156,2723.989990
2018-01-05,523.859985,446.785400,483000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008520,2.476,9.22,106.027863,139.501175,108.224648,7136.560059,2743.149902
2018-01-08,523.940002,450.117767,561500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007459,2.480,9.52,106.422684,139.048706,108.435471,7157.390137,2747.709961


In [66]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1446 entries, 2018-01-02 to 2023-09-29
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           1446 non-null   float64
 1   Adj Close      1446 non-null   float64
 2   Volume         1446 non-null   int64  
 3   SMA_50         1397 non-null   float64
 4   SMA_200        1247 non-null   float64
 5   MACD_12_26_9   1421 non-null   float64
 6   MACDh_12_26_9  1413 non-null   float64
 7   MACDs_12_26_9  1413 non-null   float64
 8   RSI_14         1432 non-null   float64
 9   Volatility     1426 non-null   float64
 10  Daily_Return   1445 non-null   float64
 11  interest_rate  1446 non-null   float64
 12  vix            1446 non-null   float64
 13  avy            1446 non-null   float64
 14  lh             1446 non-null   float64
 15  nxpi           1446 non-null   float64
 16  nsdq           1446 non-null   float64
 17  spy            1446 non-null   flo